In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/melting-point/sample_submission.csv
/kaggle/input/melting-point/train.csv
/kaggle/input/melting-point/test.csv


In [2]:
# Install required packages
!pip install rdkit autogluon.tabular scikit-learn==1.5.2 lightgbm xgboost catboost -q

# -----------------------------
# Complete Advanced Melting Point Prediction with Optimization
# -----------------------------

import os, math, warnings, random, time, shutil
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing
from dataclasses import dataclass
from typing import List, Tuple, Dict, Optional
from copy import deepcopy

# RDKit imports
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski
from rdkit.Chem.rdMolDescriptors import GetMorganFingerprintAsBitVect, GetMACCSKeysFingerprint

# AutoGluon
from autogluon.tabular import TabularPredictor

# Sklearn
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNetCV, RidgeCV, LassoCV
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.isotonic import IsotonicRegression

# Gradient Boosting
import lightgbm as lgb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# Optimization
from scipy.optimize import minimize, differential_evolution, dual_annealing

warnings.filterwarnings("ignore")
np.set_printoptions(suppress=True)

# -----------------------------
# Configuration
# -----------------------------
SEED = 42
N_FOLDS = 5
CV_SEEDS = [42, 123, 456]  # Multiple seeds for robustness
TARGET = "Tm"
ID_COL = "id"
SMILES_COL = "SMILES"

# -----------------------------
# Utility Classes and Functions
# -----------------------------

@dataclass
class CVResult:
    oof: np.ndarray
    test_pred: np.ndarray
    fold_metrics: List[float]
    model_name: str

def mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

# -----------------------------
# RDKit Feature Generation (FIXED)
# -----------------------------

def generate_features_from_smiles(smiles_list):
    """Generates all features for a list of SMILES strings using RDKit."""
    features = []
    for smiles in tqdm(smiles_list, desc="Processing SMILES", leave=False):
        mol = Chem.MolFromSmiles(smiles)
        
        feature_dict = {}
        if mol is not None:
            # Standard Descriptors
            for name, func in Descriptors.descList:
                try:
                    feature_dict[name] = func(mol)
                except Exception:
                    feature_dict[name] = np.nan
            
            # Morgan Fingerprint (using GetMorganFingerprintAsBitVect to avoid deprecation)
            fp_morgan = GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024)
            for i in range(1024):
                feature_dict[f"Morgan_{i}"] = int(fp_morgan.GetBit(i))
            
            # MACCS Keys
            fp_maccs = GetMACCSKeysFingerprint(mol)
            for i in range(167):  # MACCS has 167 bits
                feature_dict[f"MACCS_{i}"] = int(fp_maccs.GetBit(i))
                
            # Additional custom features - FIXED NAMES
            feature_dict['NumRotatableBonds'] = Descriptors.NumRotatableBonds(mol)
            feature_dict['NumHDonors'] = Lipinski.NumHDonors(mol)  # Fixed
            feature_dict['NumHAcceptors'] = Lipinski.NumHAcceptors(mol)  # Fixed
            feature_dict['MolLogP'] = Descriptors.MolLogP(mol)
            feature_dict['TPSA'] = Descriptors.TPSA(mol)
        else:
            # If molecule parsing fails, fill with zeros
            for name, _ in Descriptors.descList:
                feature_dict[name] = 0
            for i in range(1024):
                feature_dict[f"Morgan_{i}"] = 0
            for i in range(167):
                feature_dict[f"MACCS_{i}"] = 0
            feature_dict['NumRotatableBonds'] = 0
            feature_dict['NumHDonors'] = 0
            feature_dict['NumHAcceptors'] = 0
            feature_dict['MolLogP'] = 0
            feature_dict['TPSA'] = 0

        features.append(feature_dict)
    return features

def parallel_feature_generation(smiles_list, num_workers):
    """Splits the SMILES list and generates features in parallel."""
    chunks = np.array_split(smiles_list, num_workers)
    with multiprocessing.Pool(processes=num_workers) as pool:
        results = pool.map(generate_features_from_smiles, chunks)
    
    return [item for sublist in results for item in sublist]

# -----------------------------
# Manual Model Training Functions
# -----------------------------

def fit_lgbm(X, y, X_test):
    """Train LightGBM with cross-validation"""
    
    params = {
        'n_estimators': 3000,
        'learning_rate': 0.03,
        'num_leaves': 63,
        'max_depth': 7,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'reg_lambda': 1.0,
        'min_child_samples': 20,
        'random_state': SEED,
        'n_jobs': -1,
        'objective': 'mae',
        'metric': 'mae',
        'verbose': -1
    }
    
    oof = np.zeros(len(X))
    test_pred = np.zeros(len(X_test))
    fold_metrics = []
    
    for cv_seed in CV_SEEDS:
        kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=cv_seed)
        for fold, (tr_idx, va_idx) in enumerate(kf.split(X, y), 1):
            X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
            y_tr, y_va = y[tr_idx], y[va_idx]
            
            model = lgb.LGBMRegressor(**params)
            model.fit(
                X_tr, y_tr,
                eval_set=[(X_va, y_va)],
                callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False),
                          lgb.log_evaluation(0)],
            )
            
            va_pred = model.predict(X_va, num_iteration=model.best_iteration_)
            oof[va_idx] += va_pred / len(CV_SEEDS)
            fold_metrics.append(mae(y_va, va_pred))
            
            test_pred += model.predict(X_test, num_iteration=model.best_iteration_) / (N_FOLDS * len(CV_SEEDS))
    
    return CVResult(oof, test_pred, fold_metrics, 'LightGBM')

def fit_xgb(X, y, X_test):
    """Train XGBoost with cross-validation"""
    
    params = {
        'n_estimators': 3000,
        'learning_rate': 0.03,
        'max_depth': 7,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'reg_lambda': 1.0,
        'tree_method': 'hist',
        'random_state': SEED,
        'n_jobs': -1,
        'objective': 'reg:absoluteerror'
    }
    
    oof = np.zeros(len(X))
    test_pred = np.zeros(len(X_test))
    fold_metrics = []
    
    for cv_seed in CV_SEEDS:
        kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=cv_seed)
        for fold, (tr_idx, va_idx) in enumerate(kf.split(X, y), 1):
            X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
            y_tr, y_va = y[tr_idx], y[va_idx]
            
            model = XGBRegressor(**params)
            model.fit(
                X_tr, y_tr,
                eval_set=[(X_va, y_va)],
                eval_metric='mae',
                verbose=False,
                early_stopping_rounds=50,
            )
            
            va_pred = model.predict(X_va)
            oof[va_idx] += va_pred / len(CV_SEEDS)
            fold_metrics.append(mae(y_va, va_pred))
            
            test_pred += model.predict(X_test) / (N_FOLDS * len(CV_SEEDS))
    
    return CVResult(oof, test_pred, fold_metrics, 'XGBoost')

def fit_catboost(X, y, X_test):
    """Train CatBoost with cross-validation"""
    
    params = {
        'depth': 7,
        'learning_rate': 0.03,
        'n_estimators': 3000,
        'loss_function': 'MAE',
        'random_seed': SEED,
        'l2_leaf_reg': 3.0,
        'subsample': 0.8,
        'verbose': 0,
        'allow_const_label': True,
    }
    
    oof = np.zeros(len(X))
    test_pred = np.zeros(len(X_test))
    fold_metrics = []
    
    for cv_seed in CV_SEEDS:
        kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=cv_seed)
        for fold, (tr_idx, va_idx) in enumerate(kf.split(X, y), 1):
            X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
            y_tr, y_va = y[tr_idx], y[va_idx]
            
            model = CatBoostRegressor(**params)
            model.fit(
                X_tr, y_tr,
                eval_set=[(X_va, y_va)],
                early_stopping_rounds=50,
                use_best_model=True
            )
            
            va_pred = model.predict(X_va)
            oof[va_idx] += va_pred / len(CV_SEEDS)
            fold_metrics.append(mae(y_va, va_pred))
            
            test_pred += model.predict(X_test) / (N_FOLDS * len(CV_SEEDS))
    
    return CVResult(oof, test_pred, fold_metrics, 'CatBoost')

# -----------------------------
# Advanced Optimization
# -----------------------------

class EnhancedOptimizer:
    """Multi-strategy optimizer for ensemble weights"""
    
    def __init__(self, oof_preds, y_true):
        self.oof_preds = oof_preds
        self.y_true = y_true
        self.n_models = oof_preds.shape[1]
        
    def objective(self, weights):
        """Objective function to minimize"""
        weights = np.abs(weights) / (np.abs(weights).sum() + 1e-10)
        pred = (self.oof_preds * weights).sum(1)
        return mae(self.y_true, pred)
    
    def optimize_all_methods(self):
        """Try multiple optimization methods"""
        results = {}
        
        # 1. Differential Evolution
        print("Running Differential Evolution...")
        de_result = differential_evolution(
            self.objective,
            [(0, 1)] * self.n_models,
            seed=SEED,
            maxiter=100,
            workers=-1
        )
        de_weights = de_result.x / de_result.x.sum()
        results['DE'] = (de_weights, de_result.fun)
        
        # 2. Scipy minimize (Nelder-Mead)
        print("Running Nelder-Mead...")
        x0 = np.ones(self.n_models) / self.n_models
        nm_result = minimize(
            self.objective,
            x0,
            method='Nelder-Mead',
            options={'maxiter': 1000}
        )
        nm_weights = nm_result.x / nm_result.x.sum()
        results['NM'] = (nm_weights, nm_result.fun)
        
        # 3. Simple average
        avg_weights = np.ones(self.n_models) / self.n_models
        avg_score = self.objective(avg_weights)
        results['AVG'] = (avg_weights, avg_score)
        
        # Find best
        best_method = min(results.keys(), key=lambda k: results[k][1])
        best_weights, best_score = results[best_method]
        
        return best_weights, best_score, best_method

# -----------------------------
# Main Pipeline
# -----------------------------

def main():
    """Main optimization pipeline"""
    
    start_time = time.time()
    
    # Load data
    print("Loading data...")
    train_df = pd.read_csv("/kaggle/input/melting-point/train.csv")
    test_df = pd.read_csv("/kaggle/input/melting-point/test.csv")
    sample_submission = pd.read_csv("/kaggle/input/melting-point/sample_submission.csv")
    
    print(f"Train shape: {train_df.shape}")
    print(f"Test shape: {test_df.shape}")
    
    # Generate RDKit features
    print("\nGenerating RDKit features...")
    num_cores = min(4, multiprocessing.cpu_count())  # Limit cores to avoid memory issues
    print(f"Using {num_cores} CPU cores")
    
    train_features_list = parallel_feature_generation(train_df[SMILES_COL].tolist(), num_cores)
    test_features_list = parallel_feature_generation(test_df[SMILES_COL].tolist(), num_cores)
    
    train_rdkit_features = pd.DataFrame(train_features_list).fillna(0)
    test_rdkit_features = pd.DataFrame(test_features_list).fillna(0)
    
    # Combine features
    train_features = pd.concat([
        train_df.drop(columns=[SMILES_COL, ID_COL, TARGET]),
        train_rdkit_features
    ], axis=1)
    
    test_features = pd.concat([
        test_df.drop(columns=[SMILES_COL, ID_COL]),
        test_rdkit_features
    ], axis=1)
    
    # Ensure same columns
    common_cols = list(set(train_features.columns) & set(test_features.columns))
    train_features = train_features[common_cols]
    test_features = test_features[common_cols]
    
    print(f"Total features: {train_features.shape[1]}")
    
    # Target
    y_train = train_df[TARGET].values
    
    # AutoGluon training
    print("\n=== Training AutoGluon ===")
    train_ag = pd.concat([train_features, train_df[[TARGET]]], axis=1)
    
    predictor = TabularPredictor(
        label=TARGET,
        problem_type='regression',
        eval_metric='mae',
        path='AutogluonModels'
    ).fit(
        train_data=train_ag,
        time_limit=3600*2,  # 2 hours for AutoGluon
        presets='best_quality',
        num_bag_folds=5,
        num_bag_sets=1,
        num_stack_levels=1
    )
    
    # Get AutoGluon predictions
    ag_oof = predictor.predict(train_features, as_pandas=False)
    ag_test = predictor.predict(test_features, as_pandas=False)
    
    print("\n=== Training Manual Models ===")
    
    # Train manual models
    models_results = []
    
    print("Training LightGBM...")
    lgb_result = fit_lgbm(train_features, y_train, test_features)
    models_results.append(lgb_result)
    print(f"  LightGBM CV MAE: {np.mean(lgb_result.fold_metrics):.4f}")
    
    print("Training XGBoost...")
    xgb_result = fit_xgb(train_features, y_train, test_features)
    models_results.append(xgb_result)
    print(f"  XGBoost CV MAE: {np.mean(xgb_result.fold_metrics):.4f}")
    
    print("Training CatBoost...")
    cat_result = fit_catboost(train_features, y_train, test_features)
    models_results.append(cat_result)
    print(f"  CatBoost CV MAE: {np.mean(cat_result.fold_metrics):.4f}")
    
    # Combine all predictions
    all_oof = [ag_oof] + [r.oof for r in models_results]
    all_test = [ag_test] + [r.test_pred for r in models_results]
    
    oof_matrix = np.column_stack(all_oof)
    test_matrix = np.column_stack(all_test)
    
    model_names = ['AutoGluon', 'LightGBM', 'XGBoost', 'CatBoost']
    
    # Print individual scores
    print("\n=== Individual Model MAEs ===")
    for i, name in enumerate(model_names):
        score = mae(y_train, oof_matrix[:, i])
        print(f"{name:15s}: {score:.5f}")
    
    # Optimization
    print("\n=== Ensemble Optimization ===")
    optimizer = EnhancedOptimizer(oof_matrix, y_train)
    best_weights, best_score, best_method = optimizer.optimize_all_methods()
    
    print(f"\nBest optimization method: {best_method}")
    print(f"Best ensemble MAE: {best_score:.5f}")
    print("\nOptimal weights:")
    for name, weight in zip(model_names, best_weights):
        print(f"  {name:15s}: {weight:.4f}")
    
    # Generate final predictions
    test_pred = (test_matrix * best_weights).sum(1)
    
    # Post-processing
    print("\n=== Post-processing ===")
    
    # Isotonic Regression
    oof_blend = (oof_matrix * best_weights).sum(1)
    iso = IsotonicRegression(out_of_bounds='clip')
    iso.fit(oof_blend, y_train)
    test_pred_calibrated = iso.predict(test_pred)
    
    # Clipping to training range
    lower_bound = np.percentile(y_train, 0.5)
    upper_bound = np.percentile(y_train, 99.5)
    test_pred_final = np.clip(test_pred_calibrated, lower_bound, upper_bound)
    
    print(f"Prediction range: [{test_pred_final.min():.2f}, {test_pred_final.max():.2f}]")
    print(f"Training target range: [{y_train.min():.2f}, {y_train.max():.2f}]")
    
    # Create submission
    submission = sample_submission.copy()
    submission[TARGET] = test_pred_final
    submission.to_csv("submission_optimized.csv", index=False)
    
    print("\n✓ Submission saved to submission_optimized.csv")
    
    # Cleanup
    try:
        shutil.rmtree("AutogluonModels")
        print("✓ Cleaned up AutoGluon models")
    except:
        pass
    
    end_time = time.time()
    print(f"\nTotal execution time: {(end_time - start_time) / 60:.2f} minutes")
    
    return submission

if __name__ == "__main__":
    submission = main()
    print("\n=== Optimization complete! ===")
    print(submission.head())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.3/487.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 4.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
Loading data...
Train shape: (2662, 427)
Test shape: (666, 426)

Generating RDKit features...
Using 4 CPU cores


Processing SMILES:   0%|          | 0/666 [00:00<?, ?it/s][15:54:38] DEPRECATION WARNING: please use MorganGenerator
[15:54:38] DEPRECATION WARNING: please use MorganGenerator
[15:54:38] DEPRECATION WARNING: please use MorganGenerator
[15:54:38] DEPRECATION WARNING: please use MorganGenerator
[15:54:38] DEPRECATION WARNING: please use MorganGenerator
[15:54:38] DEPRECATION WARNING: please use MorganGenerator
[15:54:38] DEPRECATION WARNING: please use MorganGenerator
[15:54:38] DEPRECATION WARNING: please use MorganGenerator
[15:54:38] DEPRECATION WARNING: please use MorganGenerator
[15:54:38] DEPRECATION WARNING: please use MorganGenerator
[15:54:38] DEPRECATION WARNING: please use MorganGenerator
[15:54:38] DEPRECATION WARNING: please use MorganGenerator
[15:54:38] DEPRECATION WARNING: please use MorganGenerator
Processing SMILES:   1%|          | 4/665 [00:00<00:19, 34.46it/s][15:54:38] DEPRECATION WARNING: please use MorganGenerator
[15:54:38] DEPRECATION WARNING: please use MorganG

Total features: 1832

=== Training AutoGluon ===


		Context path: "/kaggle/working/AutogluonModels/ds_sub_fit/sub_fit_ho"
Running DyStack sub-fit ...
Beginning AutoGluon training ... Time limit = 1799s
AutoGluon will save models to "/kaggle/working/AutogluonModels/ds_sub_fit/sub_fit_ho"
Train Data Rows:    2366
Train Data Columns: 1832
Label Column:       Tm
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    30242.29 MB
	Train Data (Original)  Memory Usage: 33.07 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 1367 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generator

[1000]	valid_set's l1: 26.9617
[1000]	valid_set's l1: 24.6773
[1000]	valid_set's l1: 27.459
[1000]	valid_set's l1: 29.3576


...

[1000]	valid_set's l1: 28.4643
[2000]	valid_set's l1: 27.5831
[3000]	valid_set's l1: 27.3028
[4000]	valid_set's l1: 27.1302
[5000]	valid_set's l1: 26.968


	Ran out of time, early stopping on iteration 5125. Best iteration is:
	[5125]	valid_set's l1: 26.9535


[1000]	valid_set's l1: 29.0926
[2000]	valid_set's l1: 28.1946
[3000]	valid_set's l1: 27.7051
[4000]	valid_set's l1: 27.5257
[5000]	valid_set's l1: 27.3605


	Ran out of time, early stopping on iteration 5989. Best iteration is:
	[5918]	valid_set's l1: 27.2935
	-27.0425	 = Validation score   (-mean_absolute_error)
	57.01s	 = Training   runtime
	0.74s	 = Validation runtime
Fitting model: NeuralNetTorch_r22_BAG_L1 ... Training model for up to 2.73s of the 1797.33s of remaining time.
	Fitting 5 child models (S1F1 - S1F5) | Fitting with SequentialLocalFoldFittingStrategy (sequential: cpus=2, gpus=0)
	Time limit exceeded... Skipping NeuralNetTorch_r22_BAG_L1.
Fitting model: XGBoost_r33_BAG_L1 ... Training model for up to 1.34s of the 1795.93s of remaining time.
	Fitting 5 child models (S1F1 - S1F5) | Fitting with SequentialLocalFoldFittingStrategy (sequential: cpus=2, gpus=0)
	Time limit exceeded... Skipping XGBoost_r33_BAG_L1.
Fitting model: ExtraTrees_r42_BAG_L1 ... Training model for up to 0.02s of the 1794.61s of remaining time.
	Time limit exceeded... Skipping ExtraTrees_r42_BAG_L1.
Fitting model: WeightedEnsemble_L2 ... Training model for 


=== Training Manual Models ===
Training LightGBM...
  LightGBM CV MAE: 27.9261
Training XGBoost...
  XGBoost CV MAE: 27.9330
Training CatBoost...
  CatBoost CV MAE: 27.6143

=== Individual Model MAEs ===
AutoGluon      : 8.83558
LightGBM       : 27.41800
XGBoost        : 27.46933
CatBoost       : 27.20029

=== Ensemble Optimization ===
Running Differential Evolution...
Running Nelder-Mead...

Best optimization method: DE
Best ensemble MAE: 8.83558

Optimal weights:
  AutoGluon      : 1.0000
  LightGBM       : 0.0000
  XGBoost        : 0.0000
  CatBoost       : 0.0000

=== Post-processing ===
Prediction range: [112.62, 611.10]
Training target range: [53.54, 897.15]

✓ Submission saved to submission_optimized.csv
✓ Cleaned up AutoGluon models

Total execution time: 142.04 minutes

=== Optimization complete! ===
     id          Tm
0  1022  349.972857
1  1146  332.219737
2    79  194.284615
3  2279  201.613043
4  1342  232.271875
